###Get access token by OAuth2

In [17]:
import requests
from requests_oauthlib import OAuth2Session
import json
import pandas as pd

client_id = "androidClient"
client_secret = "secret"
username = "username"
password = "password"

authorization_url = "https://app.atimelogger.com/oauth/token?username={0}&password={1}&grant_type=password".format(username, password)
token = requests.post(authorization_url, auth=(client_id, client_secret))
access_token = json.loads(token.text)['access_token']

### Reference Dictionary
Get a dictionary contain types' **ID, Name, Parent's name** for reference.

In [185]:
r_type = requests.get("https://app.atimelogger.com/api/v2/types", 
                 headers = {'Authorization': "bearer " + access_token})
json_type = json.loads(r_type.text)
# for d in json_type['types']:
#     if d['parent']:
#         d['parent'] = d['parent']['guid']
# data = pd.DataFrame(json_type["types"])
# types = pd.DataFrame([d for d in json_type['types'] if not d['group']]).ix[:, [4,5,2,6]]
# group = pd.DataFrame([d for d in json_type['types'] if d['group']]).ix[:, [4,5,2]]
# pd.merge(types, group, left_on = 'parent', right_on = 'guid', how = 'left')

dict_group = {d['guid']:d['name'] for d in json_type['types'] if d['group']}
dict_type = {}
dict_id = {}
for d in json_type['types']:
    if not d['group']:
        parent_name = dict_group[d['parent']['guid']] if d['parent'] != None else None
        dict_type[d['guid']] = (d['name'], parent_name)
        dict_id[d['name']] = (d['guid'], parent_name)

###Helper Function

In [220]:
def getTime(unixSecond):
    """"
    unixSecond: unix timestamp
    returns: string format of date time for unix timestamp
    """
    return str(datetime.fromtimestamp(unixSecond))

def getDelta(duration):
    """"
    duration: duration in second
    returns: string format of duration
    """
    return str(timedelta(seconds = duration)).split('.')[0]

### Today's Report
Get `today`, `yesterday` as parameters and fetch all intervals of yesterday.

In [221]:
from datetime import datetime, date, timedelta
import copy

today = date.today()
day_delta = timedelta(1)
yesterday = today - day_delta
## Following method will generate 4-hours difference with actual time
# today_s = (today - date(1970,1,1)).total_seconds()
# yesterday_s = (yesterday - date(1970,1,1)).total_seconds()
today_s = today.strftime("%s")
yesterday_s = yesterday.strftime("%s")
params = {'from': int(yesterday_s), 'to': int(today_s)}

r_today = requests.get("https://app.atimelogger.com/api/v2/intervals", 
                 params = params, headers = {'Authorization': "bearer " + access_token})
intervals = json.loads(r_today.text)['intervals']

Extract `comment` as task and calculate its time.

In [222]:
### Test the correctness of time
# interval_time = copy.deepcopy(intervals)
# for d in interval_time:
#     d['from'] = str(datetime.fromtimestamp(d['from']))
#     d['to'] = str(datetime.fromtimestamp(d['to']))
# pd.DataFrame(interval_time)

from collections import defaultdict

task = defaultdict(dict)
for d in intervals:
    if d['comment']:
        task[d['comment']]['Group'] = dict_type[d['type']['guid']][1]
        if "Time" in task[d['comment']].keys():
            task[d['comment']]['Time'] += (d['to'] - d['from'])
        else:
            task[d['comment']]['Time'] = (d['to'] - d['from'])

task_list = []
for k, v in task.items():
    task_list.append(dict(Task = k, Group = v['Group'], 
                          Time = getDelta(v['Time'])))
pd.DataFrame(task_list)

,Group,Task,Time
0,Learn,Atimelogger API today's task table,3:45:30
1,Learn,Scikit learn part 2,0:41:14
2,Wealth,Select modem and router,1:41:41
3,Career,Read LinkedIn usage,2:18:48
4,Wealth,Test used modem and router,0:48:15


In [223]:
intervals

[{u'activityGuid': u'8b32cfe2-f17e-47c5-a6b8-21fcef680cef',
  u'comment': u'',
  u'from': 1437016353,
  u'guid': u'b2d5c99a-195b-4ec9-acd5-6dfd20e62f68',
  u'to': 1437016955,
  u'type': {u'guid': u'7068bdf8-ede1-4a79-880d-25bca3190341'}},
 {u'activityGuid': u'c4d05ec7-fafe-40d3-98a2-b5988924da45',
  u'comment': u"Atimelogger API today's task table",
  u'from': 1437013793,
  u'guid': u'deda98a4-22db-434f-af38-9fb3a4975d30',
  u'to': 1437016353,
  u'type': {u'guid': u'993acbdd-88b7-476e-b59f-7a335b5b18f5'}},
 {u'activityGuid': u'8b32cfe2-f17e-47c5-a6b8-21fcef680cef',
  u'comment': u'',
  u'from': 1437013258,
  u'guid': u'73468239-58e3-49f5-965e-ec866f9d4fa0',
  u'to': 1437013793,
  u'type': {u'guid': u'7068bdf8-ede1-4a79-880d-25bca3190341'}},
 {u'activityGuid': u'c4d05ec7-fafe-40d3-98a2-b5988924da45',
  u'comment': u"Atimelogger API today's task table",
  u'from': 1437009695,
  u'guid': u'cfe7a736-ec04-4065-9810-cb130d646a01',
  u'to': 1437013257,
  u'type': {u'guid': u'993acbdd-88b7-476

In [226]:
import numpy as np
sleep_id = dict_id['Sleep'][0]
rest_id = dict_id['Rest'][0]
meal_id = dict_id["Meal"][0]
rest = np.array([])
for d in intervals:
    if d['type']['guid'] == sleep_id:
        sleep = [getTime(d['from']), 
                 getTime(d['to']),
                 getDelta(d['to'] - d['from'])]
    elif d['type']['guid'] == rest_id:
        rest = np.append(rest, d['to'] - d['from'])
print getDelta(rest.min()), getDelta(rest.mean()), getDelta(rest.max())

0:01:00 0:06:22 0:10:24


### Get access token by basic authorization

In [ ]:
from requests.auth import HTTPBasicAuth
import time

# r = requests.get('https://app.atimelogger.com', auth=HTTPBasicAuth(username, password))
today = datetime.date.today()
day_delta = datetime.timedelta(1)
yesterday = today - day_delta
today_s = (today - datetime.date(1970,1,1)).total_seconds()
print today_s
yesterday_s = (yesterday - datetime.date(1970,1,1)).total_seconds()
data = {'from': int(yesterday_s), 'to': int(today_s)}
r = requests.get("https://app.atimelogger.com/api/v2/intervals", params = data, auth=HTTPBasicAuth(username, password))
print(json.dumps(json.loads(r.text), indent=2))
# print int(time.time())
# print time.gmtime()